In [4]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

### Use the following resource

https://www.zeldadungeon.net/wiki/Tears_of_the_Kingdom_Materials

----

- Get a list of all materials
    - include href links that point to additional pages

In [6]:
### each of the materials groups has a header indicated by the following: 
# -> span class="mw-headline"
# 
# Under this header, there are lists of multiple 
# 
totk_materials_url = 'https://www.zeldadungeon.net/wiki/Tears_of_the_Kingdom_Materials'
r = requests.get(totk_materials_url)
r.status_code

200

In [176]:
# find html class mw-headline in html document soup
soup = BeautifulSoup(r.text)

In [187]:
# find Contents Index
toc_list = soup.find_all('div', {"class":"toc", "id":"toc"})
# find all material headers
hdr_list = soup.find_all('h2')
# find all materials under each header
span_list = soup.find_all('span', {"class":"mw-headline"})
# get all links for each material
dl_list = soup.find_all('dl')

In [186]:
full_toc = toc_ul_list[0]
group_hdr_list = full_toc.find_all('li', {"class":"toclevel-1"})
len(group_hdr_list)


### for each materials group, find all materials that align with this group
toclevel1_pat = re.compile("toclevel-1*")
toclevel2_pat = re.compile("toclevel-2*")
material_group_dict = {}
for g in group_hdr_list:
    material_group = g.contents[0].text
    print(material_group)
    group_vals_list = g.find_all('li', {"class":toclevel2_pat})
    material_list = []
    for gv in group_vals_list:
        try:
            g_a_val = gv.find("a")
            material_name = g_a_val.find_all('span')[1].text
            material_list.append(material_name)
            print('\t'+material_name)
        except:
            print('Could not parse the above')
    if pd.isnull(material_group)==False:
        material_group_dict.update({material_group:material_list})

In [ ]:
for mat_gname, mat_list  in material_group_dict.items():
    print(mat_gname)
    print('\n')
    print('\n\t'.join(mat_list))
    print('-'*30)

In [183]:
span_vals = [s.text for s in span_list]
span_df = pd.DataFrame(span_vals)

In [198]:
dl_raw_links = [d.find('a').get('href') for d in dl_list if d.find('a') is not None]

In [202]:
base_url = 'https://www.zeldadungeon.net'
material_proper_link = [base_url+dl for dl in dl_raw_links]

['https://www.zeldadungeon.net/wiki/Golden_Apple',
 'https://www.zeldadungeon.net/wiki/Palm_Fruit',
 'https://www.zeldadungeon.net/wiki/Hylian_Tomato',
 'https://www.zeldadungeon.net/wiki/Apple',
 'https://www.zeldadungeon.net/wiki/Wildberry',
 'https://www.zeldadungeon.net/wiki/Voltfruit',
 'https://www.zeldadungeon.net/wiki/Fleet-Lotus_Seeds',
 'https://www.zeldadungeon.net/wiki/Hydromelon',
 'https://www.zeldadungeon.net/wiki/Mighty_Bananas',
 'https://www.zeldadungeon.net/wiki/Spicy_Pepper',
 'https://www.zeldadungeon.net/wiki/Fire_Fruit',
 'https://www.zeldadungeon.net/wiki/Ice_Fruit',
 'https://www.zeldadungeon.net/wiki/Shock_Fruit',
 'https://www.zeldadungeon.net/wiki/Splash_Fruit',
 'https://www.zeldadungeon.net/wiki/Dazzlefruit',
 'https://www.zeldadungeon.net/wiki/Bomb_Flower',
 'https://www.zeldadungeon.net/wiki/Puffshroom',
 'https://www.zeldadungeon.net/wiki/Muddle_Bud',
 'https://www.zeldadungeon.net/wiki/Big_Hearty_Truffle',
 'https://www.zeldadungeon.net/wiki/Hearty_Tru